In [1]:
!pip uninstall -q torchaudio torchdata torchtext pytorch-lightning -y

In [2]:
!pip install -q -r "/kaggle/input/ee-604-cod-images/requirements.txt"
!pip install -q -U gdown

In [3]:
test_img_dir_path = '/kaggle/input/ee-604-cod-images/COD10K-v3/COD10K-v3/Test/Image'
test_gt_dir_path = '/kaggle/input/ee-604-cod-images/COD10K-v3/COD10K-v3/Test/GT_Object'

In [4]:
from PIL import Image
import torch
from torchvision import transforms
from IPython.display import display
from sklearn.metrics import jaccard_score
import numpy as np
import os
from transformers import AutoModelForImageSegmentation
from tqdm import tqdm

birefnet = AutoModelForImageSegmentation.from_pretrained('zhengpeng7/BiRefNet', trust_remote_code=True)

device = 'cuda'
torch.set_float32_matmul_precision(['high', 'highest'][0])

birefnet.to(device)
birefnet.eval()
print('BiRefNet is ready to use.')
transform_image = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def evaluate_images(test_img_dir_path, test_gt_dir_path, length_cutoff=100):
    files_image = sorted(os.listdir(test_img_dir_path))
    files_gt = sorted(os.listdir(test_gt_dir_path))
    iou_scores = []
    
    for (img, gt) in tqdm(zip(files_image[:length_cutoff], files_gt[:length_cutoff]), total=len(files_image[:length_cutoff]), desc="Evaluating Images"):
        path_img = os.path.join(test_img_dir_path, img)
        path_gt = os.path.join(test_gt_dir_path, gt)
        image = Image.open(path_img)
        
        input_image = transform_image(image).unsqueeze(0).to('cuda')
        with torch.no_grad():
            preds = birefnet(input_image)[-1].sigmoid().cpu()
        pred = preds[0].squeeze()
        pred_pil = transforms.ToPILImage()(pred)
        gt_image = Image.open(path_gt)
    
        resize_transform = transforms.Resize(pred_pil.size, antialias=True)  
        gt_resized = resize_transform(gt_image)
        pred_arr = (np.array(pred_pil) > 0.5).flatten()
        gt_arr = (np.array(gt_resized.convert('L')) > 0).flatten()
    
        iou_score = jaccard_score(gt_arr, pred_arr)
        iou_scores.append(iou_score)
    return iou_scores

iou_scores = evaluate_images(test_img_dir_path, test_gt_dir_path, 1000)
avg = sum(iou_scores) / len(iou_scores)
print(avg)

config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

BiRefNet_config.py:   0%|          | 0.00/298 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhengpeng7/BiRefNet:
- BiRefNet_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


birefnet.py:   0%|          | 0.00/91.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhengpeng7/BiRefNet:
- birefnet.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/885M [00:00<?, ?B/s]

BiRefNet is ready to use.


Evaluating Images: 100%|██████████| 1000/1000 [23:13<00:00,  1.39s/it]

0.7092463403424792
